In [18]:
from datetime import datetime as dt

import pandas as pd
from dateutil.relativedelta import relativedelta

import file_helper

In [19]:
DATE_FORMAT = '%d.%m.%Y'

In [20]:
def convert_string_to_date(string_date: str) -> dt:
    return dt.strptime(string_date, DATE_FORMAT)

def normalize_columns_in_dataframe(data: pd.DataFrame, columns: list[str] = None) -> None:
    data_columns = data[columns]
    min_values = data_columns.min()
    max_values = data_columns.max()
    data[columns] = (data_columns - min_values) / (max_values - min_values)

In [22]:
geochem_data = pd.read_excel(file_helper.get_geo_chem_file_path(), 'dubki_h_tau')

In [26]:
events_catalog_data = pd.read_csv(file_helper.get_events_catalog_file_path(), sep=';').iloc[10000:]
events_catalog_data['Date'] = pd.to_datetime(events_catalog_data['Date'], format=DATE_FORMAT)

In [27]:
date_target_offset = relativedelta(years=1)
date_target_duration = relativedelta(months=3)
starting_event_class = 13